In [1]:
!pip install pathway bokeh pandas numpy --quiet
!pip install geopy
import pathway as pw
import pandas as pd
import numpy as np
import time
import datetime
from bokeh.plotting import figure, output_notebook, show
from bokeh.layouts import column
output_notebook()


Defaulting to user installation because normal site-packages is not writeable


This is not the real Pathway package.
Visit https://pathway.com/developers/ to get Pathway.
Already tried that? Visit https://pathway.com/troubleshooting/ to get help.
Note: your platform is Windows-11-10.0.22635-SP0, your Python is CPython 3.13.3.


Loading BokehJS ...

In [2]:
!pip install geopy

Defaulting to user installation because normal site-packages is not writeable


In [3]:
from geopy.distance import geodesic

def get_distance(coord1, coord2):
    return geodesic(coord1, coord2).km  # returns distance in kilometers


In [4]:
df = pd.read_csv('dataset (1).csv')
df.head()

,ID,SystemCodeNumber,Capacity,Latitude,Longitude,Occupancy,VehicleType,TrafficConditionNearby,QueueLength,IsSpecialDay,LastUpdatedDate,LastUpdatedTime
0,0,BHMBCCMKT01,577,26.144536,91.736172,61,car,low,1,0,04-10-2016,07:59:00
1,1,BHMBCCMKT01,577,26.144536,91.736172,64,car,low,1,0,04-10-2016,08:25:00
2,2,BHMBCCMKT01,577,26.144536,91.736172,80,car,low,2,0,04-10-2016,08:59:00
3,3,BHMBCCMKT01,577,26.144536,91.736172,107,car,low,2,0,04-10-2016,09:32:00
4,4,BHMBCCMKT01,577,26.144536,91.736172,150,bike,low,2,0,04-10-2016,09:59:00


In [5]:
unique_lots = df['SystemCodeNumber'].unique()
print(unique_lots)

['BHMBCCMKT01' 'BHMBCCTHL01' 'BHMEURBRD01' 'BHMMBMMBX01' 'BHMNCPHST01'
 'BHMNCPNST01' 'Broad Street' 'Others-CCCPS105a' 'Others-CCCPS119a'
 'Others-CCCPS135a' 'Others-CCCPS202' 'Others-CCCPS8' 'Others-CCCPS98'
 'Shopping']


In [6]:

df['Timestamp'] = pd.to_datetime(df['LastUpdatedDate'].astype(str) + ' ' + df['LastUpdatedTime'].astype(str),
                                 format='%d-%m-%Y %H:%M:%S')
df = df.sort_values('Timestamp').reset_index(drop=True)


In [7]:
print(df[['LastUpdatedDate', 'LastUpdatedTime', 'Timestamp']].head())


  LastUpdatedDate LastUpdatedTime           Timestamp
0      04-10-2016        07:59:00 2016-10-04 07:59:00
1      04-10-2016        07:59:00 2016-10-04 07:59:00
2      04-10-2016        07:59:00 2016-10-04 07:59:00
3      04-10-2016        07:59:00 2016-10-04 07:59:00
4      04-10-2016        07:59:00 2016-10-04 07:59:00


<h2>Stimulating Streaming Data</h2>

In [8]:
def stream_data(df, chunk_size=14):
    for i in range(0, len(df), chunk_size):
        yield df.iloc[i:i+chunk_size]
        time.sleep(2) 

stream = stream_data(df)

<h2>Model1</h2>

In [9]:
BASE_PRICE = 10
ALPHA = 2.0

def linear_price(prev_price, Occupancy, Capacity):
    return round(prev_price + ALPHA * (Occupancy / Capacity), 2)

<h2>Model2 Demand Based Pricing</h2>

In [10]:
def traffic_level(traffic_str):
    levels = {
        'low': 1,
        'medium': 2,
        'high': 3
    }
    return levels.get(traffic_str, 2)

In [11]:
def compute_demand(row, weights):
    occupancy_ratio = row['Occupancy'] / row['Capacity']
    traffic = traffic_level(row['TrafficConditionNearby'])
    return (
        weights['alpha'] * occupancy_ratio +
        weights['beta'] * row['QueueLength'] -
        weights['gamma'] * traffic+
        weights['delta'] * row['IsSpecialDay'] +
        weights['epsilon'] * vehicle_weight(row['VehicleType'])
    )

def vehicle_weight(vtype):
    return {'car': 1, 'bike': 0.5, 'truck': 1.5}.get(vtype, 1)

def dynamic_price(demand, base_price=BASE_PRICE, lamb=0.1):
    norm_demand = np.clip(demand, 0, 10) / 10
    price = base_price * (1 + lamb * norm_demand)
    return round(np.clip(price, 0.5 * base_price, 2 * base_price), 2)

weights = {
    'alpha': 2.0,
    'beta': 1.0,
    'gamma': 1.0,
    'delta': 2.0,
    'epsilon': 1.0
}


<h2>Model3:Competitive Model Pricing</h2>

In [12]:
from collections import defaultdict

BASE_PRICE = 10
proximity_threshold_km = 0.5  

# Map: lot_id → (lat, lon)
lot_coords = {row['SystemCodeNumber']: (row['Latitude'], row['Longitude']) for _, row in df.iterrows()}

# Map: lot_id → list of neighboring lot_ids
lot_neighbors = defaultdict(list)

for lot1, coord1 in lot_coords.items():
    for lot2, coord2 in lot_coords.items():
        if lot1 != lot2:
            dist = get_distance(coord1, coord2)
            if dist <= proximity_threshold_km:
                lot_neighbors[lot1].append(lot2)


In [13]:
def competition_price(current_lot_id, current_occ, lot_occupancies, base_price=BASE_PRICE):
    neighbors = lot_neighbors[current_lot_id]
    if not neighbors:
        return base_price  # no nearby lots

    neighbor_occs = [lot_occupancies[lot] for lot in neighbors if lot in lot_occupancies]

    if not neighbor_occs:
        return base_price
    
    avg_occ = sum(neighbor_occs) / len(neighbor_occs)
    adjustment = (avg_occ - 50) / 100
    price = base_price * (1 + 0.5 * adjustment)

    return round(max(price, 5), 2)


<h2>Visualising</h2>

In [14]:
from bokeh.models import HoverTool
from bokeh.models import ColumnDataSource
from bokeh.palettes import Category20
from IPython.display import clear_output
from IPython.display import display
from bokeh.plotting import figure, show

from collections import defaultdict
BASE_PRICE = 10
prev_prices = defaultdict(lambda: BASE_PRICE)     
price_history = defaultdict(list)  

def update_and_plot(prices_df):
    clear_output(wait=True) 
    p = figure(
        title="Dynamic Parking Lot Prices Over Time",
        x_axis_label='Time Step',
        y_axis_label='Price (₹)',
        width=1000,
        height=500,
        tools="pan,wheel_zoom,box_zoom,reset,save",
        toolbar_location="above",
        background_fill_color="#f7f7f7",
        border_fill_color="#ffffff"
    )

    p.title.text_font_size = "18pt"
    p.axis.axis_label_text_font_size = "12pt"
    p.axis.major_label_text_font_size = "10pt"
    p.grid.grid_line_dash = "dotted"
    p.grid.grid_line_alpha = 0.3
    p.legend.label_text_font_size = '10pt'



    color_list = Category20[20] 
    for i, (lot_id, price_series) in enumerate(prices_df.items()):
        color = color_list[i % len(color_list)] 
        x_vals = list(range(len(price_series)))
        p.line(x_vals, price_series, legend_label=f"Lot {lot_id}", line_width=2, line_color=color)

    p.legend.title = "Parking Lots"
    p.legend.location = "top_left"
    p.legend.click_policy = "hide"
    show(p)


In [15]:
def plot_competitor_comparison(price_history, step, lot_id, lot_neighbors):
    from bokeh.models import ColumnDataSource

    neighbors = lot_neighbors.get(lot_id, [])
    if not neighbors:
        print(f"No neighbors found for {lot_id}")
        return

    lots = [lot_id] + neighbors
    prices = []
    for lot in lots:
        history = price_history.get(lot, [])
        prices.append(history[step] if step < len(history) else 0)

    source = ColumnDataSource(data=dict(lots=lots, prices=prices))

    p = figure(x_range=lots, title=f"Competitor Price Comparison at Step {step}",
               width=800, height=400, toolbar_location=None, tools="")

    p.vbar(x='lots', top='prices', width=0.6, source=source, color="skyblue")
    p.xgrid.grid_line_color = None
    p.y_range.start = 0
    p.yaxis.axis_label = "Price (₹)"
    p.xaxis.major_label_orientation = 1

    show(p)


In [16]:
BASE_PRICE = 10
step = 0
prev_prices = defaultdict(lambda: BASE_PRICE)
price_history = defaultdict(list)
lot_occupancies = defaultdict(int)
# Optional: for comparison later
model1_history = defaultdict(list)
model2_history = defaultdict(list)
model3_history = defaultdict(list)

for chunk in stream:
    for _, row in chunk.iterrows():
        lot_id = row['SystemCodeNumber']
        occ = row['Occupancy']
        cap = row['Capacity']

        # --- Model 1: Linear Pricing ---
        linear = linear_price(prev_prices[lot_id], occ, cap)
        model1_history[lot_id].append(linear)

        # --- Model 2: Demand-Based Pricing ---
        demand = compute_demand(row, weights)
        dynamic = dynamic_price(demand)
        model2_history[lot_id].append(dynamic)

        # --- Model 3: Competition-Based Pricing ---
        lot_occupancies[lot_id] = occ
        comp_price = competition_price(lot_id, occ, lot_occupancies)
        model3_history[lot_id].append(comp_price)

        #Final pricing used for main graph (Model 3)
        price_history[lot_id].append(comp_price)
        prev_prices[lot_id] = comp_price

    # Real-time pricing line plot
    update_and_plot(price_history)

    #Competitor price comparison
    plot_competitor_comparison(
        price_history,
        step=step,
        lot_id="BHMBCCMKT01", 
        lot_neighbors=lot_neighbors
    )

    step += 1
    time.sleep(1)


C:\Users\91638\AppData\Local\Temp\ipykernel_14352\21124217.py:32: UserWarning: 
You are attempting to set `plot.legend.label_text_font_size` on a plot that has zero legends added, this will have no effect.

Before legend properties can be set, you must add a Legend explicitly, or call a glyph method with a legend parameter set.

  p.legend.label_text_font_size = '10pt'


KeyboardInterrupt: 